# Create a latent representation of audio

## Loading the audio

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting up the coding environment

### Installing external packages

In [ ]:
!pip install openl3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 24.7 MB/s eta 0:00:00
  Created wheel for openl3: filename=openl3-0.4.1-py2.py3-none-any.whl size=249325522 sha256=9e64c81b4d3aed9436435b24fc279caa42350d579af253967e4dc0e3d4303912
  Stored in directory: /root/.cache/pip/wheels/f3/88/93/33a947b53255468585f1d3c869811b6967f8d20e30e8e45d27
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=2f87960ed0816193955b73cb32b63ce6d210795ed9caa4676e386180320a14be
  Stored in directory: /root/.cache/pi

### Importing libraries

In [ ]:
import os
import sys
import time
import glob
import scipy.io.wavfile
import scipy.signal
import numpy as np
import pandas as pd
import librosa
import datetime
import openl3

### Defining global variables

In [ ]:
# Audio dataset location
parent_dir = '/content/drive/MyDrive/Volet1'
label_dirs = ['RE', 'BE', 'BL', 'RL']

In [ ]:
# Embedding model (trained on environmental audio)
latent_space_dim = 512
spectr = "mel128"
model = openl3.models.load_audio_embedding_model(input_repr=spectr, content_type="env",
                                                 embedding_size=latent_space_dim)

In [ ]:
# Generate latent space feature names
LATENT_SPACE = ['ac_latent_space_'+str(i) for i in range(latent_space_dim)]

## Initializing the environment 

### Getting the number of files to process per label

In [ ]:
# Get the number of audio files in each label directory
num_audio_files = {}
for label in label_dirs:
    label_dir = os.path.join(parent_dir, label)
    audio_files = glob.glob(label_dir + '/*/*.wav')
    num_audio_files[label] = len(audio_files)
num_audio_files

{'RE': 30, 'BE': 33, 'BL': 33, 'RL': 24}

### Defining auxiliary functions

In [ ]:
def date_time_parser_from_filename(filename):
    """Parse the date and time from the filename
    
    Args:
        filename (str): filename (filename format: XXX_YYYYMMDD_HHMMSS.wav)
    
    Returns:
        date_time_obj (obj): date + time
        """
    # Get the date and time from the filename
    date_time = filename.split('_')[1] + filename.split('_')[2].split('.')[0]
    # Convert the date and time to datetime object
    date_time_obj = datetime.datetime.strptime(date_time, '%Y%m%d%H%M%S')
    return date_time_obj

## Computing latent representation 



In [ ]:
# Create a dataframe to store the features
df_latent_space = pd.DataFrame(columns=['filename']+['date']+['site']+LATENT_SPACE+['label'])
# Parameters
new_sr = 16000 # new sampling rate
chunk_size = 60 # 60 seconds
total_size = 30 # 30 minutes

# Loop through the label directories and load the audio files
for label in label_dirs:
    # Print the label directory
    print('Loading label directory: {}'.format(label))
    # Get the path to the label directory
    label_dir = os.path.join(parent_dir, label)
    # Get the list of audio files
    audio_files = glob.glob(label_dir + '/*/*.wav')
    # Loop through the audio files
    for audio_file in audio_files:
        # Initialize start time
        start_time = time.time()
        # Get the filename
        filename = os.path.basename(audio_file)
        # Get the date and time from the filename
        date = date_time_parser_from_filename(filename)
        # Get the site lowercase (parent directory name)
        site = os.path.basename(os.path.dirname(audio_file)).lower()
        # Load the audio file
        audio, sr = librosa.load(audio_file)
        # Resample the audio file
        audio = librosa.resample(audio, sr, new_sr)
        # Pad the audio file to 30 minutes
        audio = np.pad(audio, (0, total_size*chunk_size*new_sr - audio.shape[0]), 'constant')
        # Compute the latent space representation of the audio file (60 seconds chunks)
        embeddings, _ = openl3.get_audio_embedding(audio, new_sr, model=model, hop_size=chunk_size, verbose=1)
        # Add the features to the dataframe
        for i in range(embeddings.shape[0]):
            # Get the latent space representation
            embedding = embeddings[i]
            # Add the features to the dataframe
            df_latent_space = df_latent_space.append(pd.DataFrame([[filename, date, site]+embedding.tolist()+[label]], columns=['filename']+['date']+['site']+LATENT_SPACE+['label']))
        # Calculate and print the total time taken for the loop
        end_time = time.time()
        total_time = end_time - start_time
        # Print the progress of the loop compare to the total number of audio files
        print('Loading audio file: {}/{} Total time taken: {} seconds'.format(audio_files.index(audio_file)+1, num_audio_files[label], total_time))

Loading label directory: RE
1/1 [==============================] - 8s 8s/step
Loading audio file: 1/30 Total time taken: 207.15117168426514 seconds
1/1 [==============================] - 7s 7s/step
Loading audio file: 2/30 Total time taken: 192.82018899917603 seconds
1/1 [==============================] - 9s 9s/step
Loading audio file: 3/30 Total time taken: 196.30777168273926 seconds
1/1 [==============================] - 8s 8s/step
Loading audio file: 4/30 Total time taken: 192.67963290214539 seconds
1/1 [==============================] - 9s 9s/step
Loading audio file: 5/30 Total time taken: 189.6624538898468 seconds
1/1 [==============================] - 8s 8s/step
Loading audio file: 6/30 Total time taken: 193.90763115882874 seconds
1/1 [==============================] - 7s 7s/step
Loading audio file: 7/30 Total time taken: 189.59312772750854 seconds
1/1 [==============================] - 7s 7s/step
Loading audio file: 8/30 Total time taken: 188.33304619789124 seconds
1/1 [========

## Saving the dataset

In [ ]:
# Save the dataframe to a csv file
path = '/content/drive/MyDrive/dataset'
dataset_filename = 'ac_latent_space' + '_' + spectr + '_' + str(latent_space_dim) + '.csv'
df_latent_space.to_csv(os.path.join(path, dataset_filename), index=False)